In [22]:
import pandas as pd
import math
import numpy as np
import torch
from sklearn.model_selection import train_test_split

In [23]:
df = pd.read_excel("./data.xlsx")
df.columns

Index(['id', 'pair', 'coin', 'coin_address', 'buying_price', 'selling_price',
       'buying_coin_age', 'selling_coin_age', 'buying_transactions_buys_m5',
       'buying_transactions_sells_m5', 'buying_transactions_buys_h1',
       'buying_transactions_sells_h1', 'buying_total_transfers',
       'buying_total_transactions', 'selling_transactions_buys_m5',
       'selling_transactions_sells_m5', 'selling_transactions_buys_h1',
       'selling_transactions_sells_h1', 'buying_volume_m5',
       'selling_volume_m5', 'buying_volume_h1', 'selling_volume_h1',
       'buying_price_change_m5', 'buying_price_change_h1',
       'selling_price_change_m5', 'selling_price_change_h1',
       'buying_liquidity', 'selling_liquidity', 'buying_fdv', 'selling_fdv',
       'buying_market_cap', 'selling_market_cap', 'is_telegram', 'is_twitter',
       'is_website', 'opening_date', 'closing_date', 'PNL', 'status', 'type'],
      dtype='object')

In [24]:
drop_columns = [
    "id",
    "pair",
    "coin",
    "coin_address", 
    "selling_price", 
    "selling_coin_age", 
    "buying_transactions_buys_h1", 
    "buying_transactions_sells_h1", 
    "selling_transactions_buys_m5", 
    "selling_transactions_sells_m5", 
    "selling_transactions_buys_h1", 
    "selling_transactions_sells_h1", 
    "selling_volume_m5", 
    "buying_volume_h1", 
    "selling_volume_h1", 
    "buying_price_change_h1",
    "selling_price_change_m5", 
    "selling_price_change_h1", 
    "selling_liquidity", 
    "selling_fdv", 
    "selling_market_cap", 
    "opening_date", 
    "closing_date", 
    "status", 
    "type"]
df = df.drop(drop_columns, axis = 1)
df.head()

,buying_price,buying_coin_age,buying_transactions_buys_m5,buying_transactions_sells_m5,buying_total_transfers,buying_total_transactions,buying_volume_m5,buying_price_change_m5,buying_liquidity,buying_fdv,buying_market_cap,is_telegram,is_twitter,is_website,PNL
0,"0,00002017","2,997760866666667",210,22,245.0,420,3402,109,"2728,23","20178,0","20178,0",1,0,1,"-97,38"
1,"0,00001977","2,546277566666667",171,17,146.0,342,297453,77,"2713,78","19773,0","19773,0",0,0,0,"-96,64"
2,"0,00002215","2,9628692666666665",178,25,279.0,356,17154,99,"2865,3","22160,0","22160,0",0,0,0,"69,71"
3,"0,00005371","2,2864479666666666",112,29,1.0,224,15360,1507,"7144,49","53716,0","53716,0",1,1,1,"-31,15"
4,"0,00007639","1,1391634333333334",65,14,409.0,130,4787,24,"27985,21","76391,0","76391,0",0,0,0,"74,76"


In [25]:
renamed_columns = {
    "buying_price": "price", 
    "buying_coin_age": "coin_age", 
    "buying_transactions_buys_m5": "buys_m5",
    "buying_transactions_sells_m5": "sells_m5", 
    "buying_total_transfers": "total_transfers",
    "buying_total_transactions": "total_transactions", 
    "buying_volume_m5": "volume_m5",
    "buying_price_change_m5": "price_change_m5",
    "buying_liquidity": "liquidity", 
    "buying_fdv": "fdv",
    "buying_market_cap": "market_cap"
}
df.rename(columns=renamed_columns, inplace=True) 
df

,price,coin_age,buys_m5,sells_m5,total_transfers,total_transactions,volume_m5,price_change_m5,liquidity,fdv,market_cap,is_telegram,is_twitter,is_website,PNL
0,"0,00002017","2,997760866666667",210,22,245.0,420,3402,109,"2728,23","20178,0","20178,0",1,0,1,"-97,38"
1,"0,00001977","2,546277566666667",171,17,146.0,342,297453,77,"2713,78","19773,0","19773,0",0,0,0,"-96,64"
2,"0,00002215","2,9628692666666665",178,25,279.0,356,17154,99,"2865,3","22160,0","22160,0",0,0,0,"69,71"
3,"0,00005371","2,2864479666666666",112,29,1.0,224,15360,1507,"7144,49","53716,0","53716,0",1,1,1,"-31,15"
4,"0,00007639","1,1391634333333334",65,14,409.0,130,4787,24,"27985,21","76391,0","76391,0",0,0,0,"74,76"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,"0,0000822","1,1618589166666666",33,9,3365.0,66,14014,37,"28354,47","82201,0","82201,0",1,0,0,"78,47"
291,"0,00001556","2,8510967166666665",108,42,143.0,216,24690,1918,"6758,15","15567,0","15567,0",0,1,0,"74,04"
292,"0,0001193","2,0249139166666668",127,56,4914.0,254,29917,96,"34395,14","119318,0","119318,0",0,0,0,"64,71"
293,"0,00005807","2,0921795333333333",76,51,787.0,152,13556,-4,"24014,45","58072,0","58072,0",1,1,0,"63,70"


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price               295 non-null    object 
 1   coin_age            295 non-null    object 
 2   buys_m5             295 non-null    int64  
 3   sells_m5            295 non-null    int64  
 4   total_transfers     291 non-null    float64
 5   total_transactions  295 non-null    int64  
 6   volume_m5           295 non-null    int64  
 7   price_change_m5     295 non-null    int64  
 8   liquidity           295 non-null    object 
 9   fdv                 295 non-null    object 
 10  market_cap          295 non-null    object 
 11  is_telegram         295 non-null    int64  
 12  is_twitter          295 non-null    int64  
 13  is_website          295 non-null    int64  
 14  PNL                 295 non-null    object 
dtypes: float64(1), int64(8), object(6)
memory usage: 34.7+ KB

In [27]:
df = df.dropna(subset=['total_transfers'])


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 291 entries, 0 to 294
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price               291 non-null    object 
 1   coin_age            291 non-null    object 
 2   buys_m5             291 non-null    int64  
 3   sells_m5            291 non-null    int64  
 4   total_transfers     291 non-null    float64
 5   total_transactions  291 non-null    int64  
 6   volume_m5           291 non-null    int64  
 7   price_change_m5     291 non-null    int64  
 8   liquidity           291 non-null    object 
 9   fdv                 291 non-null    object 
 10  market_cap          291 non-null    object 
 11  is_telegram         291 non-null    int64  
 12  is_twitter          291 non-null    int64  
 13  is_website          291 non-null    int64  
 14  PNL                 291 non-null    object 
dtypes: float64(1), int64(8), object(6)
memory usage: 36.4+ KB


In [29]:
df["price"] = df["price"].replace(",",".", regex=True).astype(float)
df["coin_age"] = df["coin_age"].replace(",",".", regex=True).astype(float)
df["liquidity"] = df["liquidity"].replace(",",".", regex=True).astype(float)
df["fdv"] = df["fdv"].replace(",",".", regex=True).astype(float)
df["market_cap"] = df["market_cap"].replace(",",".", regex=True).astype(float)
df["PNL"] = df["PNL"].replace(",",".", regex=True).astype(float)

/tmp/ipykernel_1492/528399640.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["price"] = df["price"].replace(",",".", regex=True).astype(float)
/tmp/ipykernel_1492/528399640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["coin_age"] = df["coin_age"].replace(",",".", regex=True).astype(float)
/tmp/ipykernel_1492/528399640.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [30]:
df.loc[df["PNL"] >= 60, "profit"] = 1
df.loc[df["PNL"] < 60, "profit"] = 0
df = df.drop(["PNL"], axis = 1)
df["profit"] = df["profit"].astype(int)

/tmp/ipykernel_1492/1008021683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df["PNL"] >= 60, "profit"] = 1


In [31]:
df = df[(df["coin_age"] < 6) & (df["liquidity"] >= 1000)]

In [32]:
df.head()

,price,coin_age,buys_m5,sells_m5,total_transfers,total_transactions,volume_m5,price_change_m5,liquidity,fdv,market_cap,is_telegram,is_twitter,is_website,profit
0,0.000020,2.997761,210,22,245.0,420,3402,109,2728.23,20178.0,20178.0,1,0,1,0
1,0.000020,2.546278,171,17,146.0,342,297453,77,2713.78,19773.0,19773.0,0,0,0,0
2,0.000022,2.962869,178,25,279.0,356,17154,99,2865.30,22160.0,22160.0,0,0,0,1
3,0.000054,2.286448,112,29,1.0,224,15360,1507,7144.49,53716.0,53716.0,1,1,1,0
4,0.000076,1.139163,65,14,409.0,130,4787,24,27985.21,76391.0,76391.0,0,0,0,1


In [33]:
df.describe()

,price,coin_age,buys_m5,sells_m5,total_transfers,total_transactions,volume_m5,price_change_m5,liquidity,fdv,market_cap,is_telegram,is_twitter,is_website,profit
count,2.840000e+02,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,2.840000e+02,284.000000,2.840000e+02,2.840000e+02,284.000000,284.000000,284.000000,284.000000
mean,6.388920e-02,2.467837,113.612676,41.126761,6028.591549,228.260563,53975.602113,1.379050e+05,60948.200775,1.291753e+08,1.291753e+08,0.116197,0.109155,0.091549,0.443662
std,9.864698e-02,0.971388,89.743321,57.799074,55537.999196,181.250863,61213.113944,3.974061e+05,73124.613346,4.808941e+08,4.808941e+08,0.321027,0.312384,0.288898,0.497693
min,5.676000e-08,0.810816,0.000000,0.000000,0.000000,0.000000,3.000000,-8.900000e+01,1087.600000,2.565000e+03,2.565000e+03,0.000000,0.000000,0.000000,0.000000
25%,7.359750e-05,1.742977,57.000000,12.000000,2.000000,114.000000,15183.500000,4.375000e+01,25445.660000,6.865075e+04,6.865075e+04,0.000000,0.000000,0.000000,0.000000
50%,1.732500e-02,2.496501,97.000000,25.000000,233.500000,194.000000,30497.500000,2.791900e+04,37698.595000,1.682876e+07,1.682876e+07,0.000000,0.000000,0.000000,0.000000
75%,9.301250e-02,3.064882,156.750000,52.250000,436.250000,313.500000,74643.000000,1.351995e+05,82371.330000,9.553306e+07,9.553306e+07,0.000000,0.000000,0.000000,1.000000
max,4.358000e-01,5.315377,688.000000,471.000000,818285.000000,1376.000000,350169.000000,4.083819e+06,509352.430000,3.830535e+09,3.830535e+09,1.000000,1.000000,1.000000,1.000000


In [34]:
x = df.drop(columns=["profit"])
y = df["profit"].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [35]:
x_train = torch.tensor(x_train.values).to(torch.float32)
x_test = torch.tensor(x_test.values).to(torch.float32)
y_train = torch.tensor(y_train.values).to(torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test.values).to(torch.float32).unsqueeze(1)

In [36]:
class LogisticRegression(torch.nn.Module):

    def __init__(self, n_features):
        super(LogisticRegression, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out

In [37]:
n_features = x_train.shape[1]
model = LogisticRegression(n_features)

optim = torch.optim.SGD(model.parameters(), lr=1)
criterion = torch.nn.BCELoss()

In [38]:
EPOCHS = 3

def train(model, optim, criterion, x, y, epochs=EPOCHS):
    for e in range(1, epochs + 1):
        optim.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optim.step()
        print(f'Среднее значение функции потерь за эпоху {e}: {loss.data}')
    return model

In [39]:
model = train(model, optim, criterion, x_train, y_train)

Среднее значение функции потерь за эпоху 1: 55.50660705566406
Среднее значение функции потерь за эпоху 2: 55.50660705566406
Среднее значение функции потерь за эпоху 3: 55.50660705566406


In [40]:
def accuracy(model, x, y):
    out = model(x)
    correct = torch.abs(y - out) < 0.5
    return correct.float().mean()

plain_accuracy = accuracy(model, x_test, y_test)
print(f'Точность модели: {plain_accuracy}')

Точность модели: 0.4385964870452881


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

x = df.drop(columns=["profit"])
y = df["profit"].copy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Создание и обучение модели логистической регрессии
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Предсказание классов на тестовой выборке
y_pred = model.predict(X_test)

# Вычисление точности классификации
accuracy = accuracy_score(y_test, y_pred)
print("Точность классификации: {:.2f}%".format(accuracy * 100))

Точность классификации: 57.89%
